Notebook to add better categories to RSO data

In [3]:
import json
import asyncio
from groq import Groq
import os
from typing import Dict, List, Optional
import time
from tqdm import tqdm
from dotenv import load_dotenv


In [4]:
load_dotenv()  # This loads the variables from .env

client = Groq(
    api_key=os.environ["GROQ_API_KEY"],
)

In [5]:
VALID_CATEGORIES = [
    "Biology", "Chemistry", "Physics", "Mathematics", "Computer Science", 
    "Data Science", "Economics", "Psychology", "Sociology", "Political Science",
    "History", "Philosophy", "Literature", "Languages", "Law", "Medicine",
    "Nursing", "Public Health", "Engineering", "Environmental Science",
    "Finance", "Investment", "Quantitative Trading", "Private Equity",
    "Venture Capital", "Consulting", "Marketing", "Entrepreneurship",
    "Real Estate", "Technology", "Software Development", "Product Management",
    "Healthcare", "Legal", "Research", "Journalism", "Media Production",
    "Visual Arts", "Painting", "Photography", "Digital Art", "Music",
    "Band", "Choir", "A Cappella", "Theater", "Dance", "Film",
    "Creative Writing", "Design", "Cultural", "International", "Religious",
    "LGBTQ+", "Gender & Sexuality", "Social Justice", "Political", "Activism",
    "Community Service/volunteering", "Mentorship", "Environmental",
    "Team Sports", "Individual Sports", "E-sports", "Gaming", "Chess",
    "Debate", "Model UN", "Food & Cooking", "Travel", "Outdoor Activities",
    "Student Government", "Publications", "Mental Health", "Wellness",
    "Career Development", "Academic Support", "Leadership", "Greek Life"
]

In [6]:
FEW_SHOT_EXAMPLES = [
    {
        "name": "Investment Banking Group",
        "description": "A professional organization dedicated to educating members about investment banking, private equity, and financial markets. We host networking events, technical workshops, and mock interviews to prepare students for careers in finance.",
        "ideal_categories": ["Finance", "Investment", "Career Development"],
        "explanation": "This RSO focuses on finance education and career preparation, warranting multiple related financial categories."
    },
    {
        "name": "Data Science for Social Good",
        "description": "We apply data science and machine learning techniques to tackle social issues in healthcare, education, and environmental sustainability. Members work on real-world projects while learning technical skills.",
        "ideal_categories": ["Data Science", "Computer Science", "Community Service/volunteering"],
        "explanation": "Combines technical data science work with social impact, deserving both technical and service categories."
    },
    {
        "name": "Mental Health Alliance",
        "description": "A student organization focused on promoting mental health awareness, reducing stigma, and connecting students with resources. We organize wellness workshops, peer support groups, and educational events.",
        "ideal_categories": ["Mental Health", "Wellness", "Student Life"],
        "explanation": "Focuses on mental health and wellness within student life context."
    }
]